In [44]:
import pandas as pd
import json
from tqdm import tqdm

In [24]:
df = pd.read_csv('braun2025_peptide_rank.csv')
df.head()

,Peptide,Peptide (with PTM),Length,Accession,VMM1/Nil 1,VMM1/Nil 2,VMM1/Nil 3,VMM1/IFN 1,VMM1/IFN 2,VMM1/IFN 3,...,median_intensity,melanocyte_median_intensity,keratinocyte_median_intensity,melanocyte_detection_count,keratinocyte_detection_count,melanocyte_rank,keratinocyte_rank,gtex_rank,netMHCpan4_1,netMHCpan4_1_binding_prediction
0,TRQDHAQQL,TRQDHAQQL,9.0,F5H6Y5|F5H6Y5_HUMAN;Q14980-2|NUMA1_HUMAN;Q1498...,97100000.0,330000000.0,2.490000e+08,1.770000e+07,1.390000e+09,6.090000e+08,...,52500000.0,2.895000e+08,1760000.0,6,9,33,1,2306,0.0066,SB
1,SSGPQRLV,SSGPQRLV,8.0,J3KRW7|J3KRW7_HUMAN;Q8N511|TM199_HUMAN;J3KS81|...,27100000.0,75500000.0,2.210000e+08,8.060000e+07,3.240000e+08,5.800000e+07,...,58000000.0,7.805000e+07,7910000.0,6,9,201,3,6735,4.3740,NaN
2,SSVPGVRLL,SSVPGVRLL,9.0,B0YJC4|B0YJC4_HUMAN;P08670|VIME_HUMAN;A0A1B0GV...,143000000.0,473000000.0,2.730000e+09,1.570000e+09,6.970000e+09,4.690000e+09,...,83400000.0,2.150000e+09,3125000.0,6,8,114,23,659,0.0622,SB
3,SAYGSVKAY,SAYGSVKAY,9.0,H0YKV8|H0YKV8_HUMAN;P07355|ANXA2_HUMAN;H0YN42|...,109000000.0,374000000.0,5.430000e+08,3.130000e+08,1.710000e+09,1.240000e+09,...,103000000.0,4.585000e+08,361000.0,6,8,99,19,720,0.1412,SB
4,VRPDNTYEV,VRPDNTYEV,9.0,K7EJB9|K7EJB9_HUMAN;A0A7P0T861|A0A7P0T861_HUMA...,45200000.0,301000000.0,3.980000e+08,2.550000e+08,6.590000e+08,8.050000e+08,...,118100000.0,3.495000e+08,11095000.0,6,8,59,14,961,0.0272,SB


In [46]:
sort_gtex = df.sort_values(by='gtex_rank', ascending=True).head(300)
peptides = [pep for pep in list(sort_gtex['Peptide'])]

sort_mel = df[~df['Peptide'].isin(top)].sort_values(by='melanocyte_rank', ascending=True).head(300)
peptides = peptides + [pep for pep in list(sort_mel['Peptide'])]

sort_ker = df[~df['Peptide'].isin(top)].sort_values(by='keratinocyte_rank', ascending=True).head(300)
peptides = peptides + [pep for pep in list(sort_ker['Peptide'])]

print(len(top))

900


In [42]:
with open('/home/anushka.agrawal/hla_alignment/H_sapiens_codon_usage.json', 'r') as infile:
    codon_freq_dict = json.load(infile)

In [47]:
# taken from tcraft code
def dfs_seq_builder(ref_aa, primer='', aa_pos=0):
    """
    This function uses a depth-first search to build all possible DNA sequences that translate to the given amino acid sequence.

    params:
        ref_aa: str, the reference amino acid sequence
        primer: str, the current DNA sequence being built
        aa_pos: int, the current position in the amino acid sequence

    returns:
        list of all possible DNA sequences that translate to the given amino acid sequence
    """
    if aa_pos >= len(ref_aa):
        return [primer]
    output = []
    for codon_option, _ in codon_freq_dict[ref_aa[aa_pos]]:
        output.extend(dfs_seq_builder(ref_aa, primer+codon_option, aa_pos+1))
    return output 

In [48]:
# filter out low frequency codons
codons = []
for aa, codon_list in codon_freq_dict.items():
    codon_freq_dict[aa] = [codon for codon in codon_list if codon[1] >= 0.1]
    for codon in codon_list:
        codons.append(codon[0])

# find every possible nt sequence for each peptide
def build_sequence_dict(peptide_list):
    sequence_dict = dict()
    for peptide in tqdm(peptide_list):
        sequence_dict[peptide] = dfs_seq_builder(peptide)
    return sequence_dict

sequence_dict = build_sequence_dict([peptide for peptide in peptides])

100%|██████████| 900/900 [01:56<00:00,  7.74it/s]


In [54]:
peptide_list = [(pep, seqs[0]) for pep, seqs in sequence_dict.items()]

with open('peptides_for_idt.txt', 'w') as file:
    for peptide, sequence in peptide_list:
        file.write(f"{peptide},{sequence}\n")